In [2]:
import glob
import io

import polars as pl 
from google.cloud import bigquery

import pandas as pd

In [16]:
def extract_new_trades():
    file_paths = glob.glob("/Users/manishvasu/Downloads/tradebook*.csv")
    df = pl.read_csv(file_paths[0])

    df = df.select(pl.col("symbol"),
                   pl.col("isin"),
                   pl.col("trade_date").str.to_date(format="%d/%m/%y"),
                   pl.col("exchange"),
                   pl.col("trade_type"),
                   pl.col("quantity").cast(pl.Int64),
                   pl.col("price"))

    df = df.with_columns(pnl=pl.lit(None).cast(pl.Float64))

    return df

In [17]:
def load_new_trades(df):
    client = bigquery.Client()

    with io.BytesIO() as stream:
        df.write_parquet(stream)
        stream.seek(0)
        job = client.load_table_from_file(
            stream,
            destination='plutus-424506.equity.equity-pnl',
            project='plutus-424506',
            job_config=bigquery.LoadJobConfig(
                source_format=bigquery.SourceFormat.PARQUET,
            ),
        )
    job.result() 

In [18]:
def extract_existing_trades():
    client = bigquery.Client()

    QUERY = (
        'SELECT * FROM `plutus-424506.equity.equity-pnl`'
        )
    query_job = client.query(QUERY)
    rows = query_job.result()

    df = pl.from_arrow(rows.to_arrow())

    return df

In [19]:
new_trades = extract_new_trades()
new_trades

symbol,isin,trade_date,exchange,trade_type,quantity,price,pnl
str,str,date,str,str,i64,f64,f64
"""DISHTV""","""INE836F01026""",2020-12-17,"""NSE""","""buy""",1750,14.1,null
"""MEGH""","""INE974H01013""",2020-12-17,"""NSE""","""buy""",105,79.7,null
"""MEGH""","""INE974H01013""",2020-12-17,"""NSE""","""buy""",245,79.7,null
"""DISHTV""","""INE836F01026""",2020-12-21,"""NSE""","""buy""",1700,13.5,null
"""MEGH""","""INE974H01013""",2020-12-21,"""NSE""","""buy""",140,79.0,null
…,…,…,…,…,…,…,…
"""HUDCO""","""INE031A01017""",2024-05-23,"""NSE""","""sell""",31,270.0,null
"""ADANIPORTS""","""INE742F01042""",2024-05-29,"""NSE""","""buy""",11,1415.0,null
"""JIOFIN""","""INE758E01017""",2024-05-29,"""NSE""","""buy""",40,350.0,null


In [23]:
test_df = new_trades.select(pl.all()).filter(pl.col("symbol") == "ENGINERSIN").sort("trade_date")
test_df

symbol,isin,trade_date,exchange,trade_type,quantity,price,pnl
str,str,date,str,str,i64,f64,f64
"""ENGINERSIN""","""INE510A01028""",2023-10-23,"""NSE""","""buy""",72,131.1,null
"""ENGINERSIN""","""INE510A01028""",2024-05-23,"""BSE""","""sell""",16,285.0,null
"""ENGINERSIN""","""INE510A01028""",2024-05-23,"""BSE""","""sell""",10,285.0,null
"""ENGINERSIN""","""INE510A01028""",2024-05-23,"""BSE""","""sell""",10,285.0,null


In [21]:
arr = [[1], [2], [3]]
x = map(id, arr)
for i in x:
    print(i)

4845936640
4845637376
4845463104


In [22]:
arr_2 = arr[0:2]
y = map(id, arr_2)
for i in y:
    print(i)
print(arr[0:2])

arr[1][0] = 5 

z = map(id, arr_2)
for i in z:
    print(i)
print(arr_2)

x = map(id, arr)
for i in x:
    print(i)

4845936640
4845637376
[[1], [2]]
4845936640
4845637376
[[1], [5]]
4845936640
4845637376
4845463104
